In [36]:
# some functions I have used for preprocessing
# I will use data from this dataset only, as an external factors
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from time import *
import os
from datetime import datetime, timedelta

In [37]:
def files_data_concat(dir):
    onlyfiles = [f for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f))]
    # df = pd.read_csv(os.path.join(dir, onlyfiles[0]), encoding='unicode_escape', sep=';')
    # onlyfiles = onlyfiles[1:]
    # for file in onlyfiles:
    #     tmp = pd.read_csv(os.path.join(dir, file), encoding='unicode_escape', sep=';')
    dfs = []
    for file in onlyfiles:
        dfs.append(pd.read_csv(os.path.join(dir, file), encoding='unicode_escape', sep=';'))
    return pd.concat(dfs)
        

In [38]:
DIR_NAME = 'additional_data'
df = files_data_concat(DIR_NAME)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9145 entries, 0 to 359
Data columns (total 17 columns):
 #   Column                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                --------------  -----  
 0   Doba                                                                                                                                                                                  9145 non-null   object 
 1   Godzina                                                                                                                                                                               9145 non-null   int64  
 2   Prognozowane zapotrzebowanie sieci                

In [40]:
df.isna().sum()

Doba                                                                                                                                                                                       0
Godzina                                                                                                                                                                                    0
Prognozowane zapotrzebowanie sieci                                                                                                                                                         0
Wymagana rezerwa mocy OSP                                                                                                                                                                  0
Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13)                                                                                                                             0
Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê m

In [41]:
df.index = np.arange(len(df.index))

In [42]:
# jak się okazuje ktoś mądry zapomniał o jednej godzinie 23 marca ale nadrobił to 29 października ufff xD
# teraz to w sumie nie wiadomo co z tym zrobić, ale potraktuję to że po prostu godziny są po kolei tylko wprowadzone daty są złe
# trzeba oczywiście zshiftować godziny pomiędzy błędem i korektą żeby były dobrze, czyli -1 w tym wypadku.....
tmp = df.groupby('Doba').size()

In [43]:
tmp = tmp.sort_values()

In [44]:
tmp

Doba
2022-11-21     1
2023-03-26    23
2023-08-06    24
2023-08-05    24
2023-08-04    24
              ..
2023-03-22    24
2023-03-21    24
2023-03-31    24
2023-12-07    24
2023-10-29    25
Length: 382, dtype: int64

In [45]:
# 1 rekord ma złą postać godziny i jest to błąd (dobra to nie błąd, zajęło mi kilka godzin ale ogarnałem że to zmiana czasu XD)
df[df['Godzina']==25]

,Doba,Godzina,Prognozowane zapotrzebowanie sieci,Wymagana rezerwa mocy OSP,Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13),Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4),Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB dostêpna dla OSP,Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9),Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB,Prognozowana sumaryczna generacja róde³ wiatrowych,Prognozowana sumaryczna generacja róde³ fotowoltaicznych,Planowane saldo wymiany miêdzysystemowej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Przewidywana generacja zasobów wytwórczych nieobjêtych obowi¹zkami mocowymi,Obowi¹zki mocowe wszystkich jednostek rynku mocy
8208,2023-10-29,25,14286,1504,4684,3180.0,20685,10 685,6001,9378,6570,0,-1093,0,0.0,12812,0


In [46]:
df.drop([8208], inplace=True)

In [47]:
# df_tmp = df.loc[3000:8207]
# df_start = df.loc[:2999]
# df_end = df.loc[8208:]

In [48]:
def Date_Fix(df):
    df['Doba'] = pd.to_datetime(df['Doba'])
    df['Doba'] = np.where(df['Godzina'] == 24, df['Doba'] + pd.Timedelta("1 day") ,df['Doba'])
    df['Godzina']  = np.where(df['Godzina'] == 24, 0 , df['Godzina'])
    return df

In [49]:
# df_tmp = Date_Fix(df_tmp)
# df_start = Date_Fix(df_start)
# df_end = Date_Fix(df_end)
df = Date_Fix(df)

In [50]:
# df_tmp["Doba_2"] = df_tmp["Doba"].astype(str) + ' ' + df_tmp["Godzina"].astype(str) + ':00:00.000000 UTC'
# df_start["Doba_2"] = df_start["Doba"].astype(str) + ' ' + df_start["Godzina"].astype(str) + ':00:00.000000 UTC'
# df_end["Doba_2"] = df_end["Doba"].astype(str) + ' ' + df_end["Godzina"].astype(str) + ':00:00.000000 UTC'
df["Doba_2"] = df["Doba"].astype(str) + ' ' + df["Godzina"].astype(str) + ':00:00.000000 UTC'

In [51]:
# df_tmp["Doba_2"] = pd.to_datetime(df_tmp["Doba_2"])
# df_start["Doba_2"] = pd.to_datetime(df_start["Doba_2"])
# df_end["Doba_2"] = pd.to_datetime(df_end["Doba_2"])
df["Doba_2"] = pd.to_datetime(df["Doba_2"])

In [52]:
#df_tmp['Doba_2'] = pd.to_datetime(df_tmp['Doba_2']) - pd.Timedelta("1 hour")
# przed ogarnieciem że jest zmiana czasu XD

In [53]:
# Let's see if the results match
# df_concat = pd.concat([df_start, df_tmp, df_end])
# df_concat = df_concat[:-1]

In [54]:
# Finally some result, I was doing this for +-5 hours
# df_concat.nunique()
df.nunique()

Doba                                                                                                                                                                                     382
Godzina                                                                                                                                                                                   24
Prognozowane zapotrzebowanie sieci                                                                                                                                                      6349
Wymagana rezerwa mocy OSP                                                                                                                                                               1367
Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13)                                                                                                                          4141
Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê m

In [55]:
# możemy usunąć ostatni rekord i póżniej dodamy jeden w marcu około 3 w nocy
df

,Doba,Godzina,Prognozowane zapotrzebowanie sieci,Wymagana rezerwa mocy OSP,Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13),Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4),Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB dostêpna dla OSP,Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9),Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB,Prognozowana sumaryczna generacja róde³ wiatrowych,Prognozowana sumaryczna generacja róde³ fotowoltaicznych,Planowane saldo wymiany miêdzysystemowej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Przewidywana generacja zasobów wytwórczych nieobjêtych obowi¹zkami mocowymi,Obowi¹zki mocowe wszystkich jednostek rynku mocy,Doba_2
0,2022-11-22,0,18031,1741,5167,NaN,18955,18 745,13578,3982,346,0,471,0,NaN,14838,0,2022-11-22 00:00:00+00:00
1,2022-11-22,1,16573,1652,3428,NaN,16082,15 872,12444,3883,322,0,247,0,NaN,13271,0,2022-11-22 01:00:00+00:00
2,2022-11-22,2,16852,1686,3533,NaN,16345,16 135,12603,3938,395,0,312,0,NaN,13200,0,2022-11-22 02:00:00+00:00
3,2022-11-22,3,16802,1689,3585,NaN,16345,16 135,12550,3996,452,0,256,0,NaN,13165,0,2022-11-22 03:00:00+00:00
4,2022-11-22,4,16837,1680,3894,NaN,16345,16 135,12241,4180,635,0,416,0,NaN,13136,0,2022-11-22 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9140,2023-12-07,20,23336,2211,5299,3087.0,22315,22 315,17016,4958,717,0,1362,0,0.0,2619,23563,2023-12-07 20:00:00+00:00
9141,2023-12-07,21,22674,2136,5942,3806.0,22315,22 315,16372,4865,735,0,1438,0,0.0,2607,23563,2023-12-07 21:00:00+00:00
9142,2023-12-07,22,21120,2005,6743,4738.0,21936,21 936,15193,4793,750,0,1134,0,0.0,2587,23563,2023-12-07 22:00:00+00:00
9143,2023-12-07,23,19736,1891,7523,5632.0,21465,21 465,13943,4690,688,0,1104,0,0.0,15728,0,2023-12-07 23:00:00+00:00


In [56]:
#df_concat.index = df_concat['Doba_2']

In [57]:
df = df.iloc[:-1,]

In [58]:
# have to add it on place 3000
df[df['Doba']=='2023-03-27']

,Doba,Godzina,Prognozowane zapotrzebowanie sieci,Wymagana rezerwa mocy OSP,Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13),Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4),Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB dostêpna dla OSP,Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9),Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB,Prognozowana sumaryczna generacja róde³ wiatrowych,Prognozowana sumaryczna generacja róde³ fotowoltaicznych,Planowane saldo wymiany miêdzysystemowej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Przewidywana generacja zasobów wytwórczych nieobjêtych obowi¹zkami mocowymi,Obowi¹zki mocowe wszystkich jednostek rynku mocy,Doba_2
3000,2023-03-27,1,13734,1396,4828,NaN,17208,12 891,8063,5671,1948,0,0,0,NaN,11165,0,2023-03-27 01:00:00+00:00
3001,2023-03-27,2,13185,1341,5468,NaN,17250,12 927,7459,5726,1972,0,0,0,NaN,10614,0,2023-03-27 02:00:00+00:00
3002,2023-03-27,3,12984,1325,6326,NaN,17253,13 364,7038,5946,2175,0,0,0,NaN,10399,0,2023-03-27 03:00:00+00:00
3003,2023-03-27,4,13070,1338,6401,NaN,17263,13 374,6974,6097,2324,0,0,0,NaN,10502,0,2023-03-27 04:00:00+00:00
3004,2023-03-27,5,13444,1378,6641,NaN,17282,13 393,6752,6692,2923,0,0,0,NaN,11213,0,2023-03-27 05:00:00+00:00
3005,2023-03-27,6,14430,1434,6912,NaN,17264,13 602,6690,7184,3383,0,556,0,NaN,11882,0,2023-03-27 06:00:00+00:00
3006,2023-03-27,7,17180,1646,5074,NaN,17480,13 818,8744,7134,3281,10,1302,0,NaN,13837,0,2023-03-27 07:00:00+00:00
3007,2023-03-27,8,19070,1845,2933,NaN,17886,14 224,11253,6855,2865,242,962,37,NaN,4253,23614,2023-03-27 08:00:00+00:00
3008,2023-03-27,9,19786,1993,2760,NaN,18974,15 312,12515,7028,2849,752,242,37,NaN,4443,23614,2023-03-27 09:00:00+00:00
3009,2023-03-27,10,19444,1941,3384,NaN,17657,14 353,10574,8055,3474,1698,816,395,NaN,5478,23614,2023-03-27 10:00:00+00:00


In [59]:
new_row = {}
for col in df.columns:
        new_row[col] = np.nan
new_row['Doba_2'] = '2023-03-27 00:00:00+00:00'
new_row['Doba'] = '2023-03-27 00:00:00'
new_row['Godzina'] = 0

In [60]:
df_1 = df.iloc[:3000]
df_2 = df.iloc[3000:]

In [61]:
df_1 = pd.concat([df_1, pd.DataFrame(new_row, index=[3000])])

In [62]:
df_1

,Doba,Godzina,Prognozowane zapotrzebowanie sieci,Wymagana rezerwa mocy OSP,Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13),Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4),Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB dostêpna dla OSP,Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9),Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB,Prognozowana sumaryczna generacja róde³ wiatrowych,Prognozowana sumaryczna generacja róde³ fotowoltaicznych,Planowane saldo wymiany miêdzysystemowej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Przewidywana generacja zasobów wytwórczych nieobjêtych obowi¹zkami mocowymi,Obowi¹zki mocowe wszystkich jednostek rynku mocy,Doba_2
0,2022-11-22 00:00:00,0,18031.0,1741.0,5167.0,NaN,18955.0,18 745,13578.0,3982.0,346.0,0.0,471.0,0.0,NaN,14838.0,0.0,2022-11-22 00:00:00+00:00
1,2022-11-22 00:00:00,1,16573.0,1652.0,3428.0,NaN,16082.0,15 872,12444.0,3883.0,322.0,0.0,247.0,0.0,NaN,13271.0,0.0,2022-11-22 01:00:00+00:00
2,2022-11-22 00:00:00,2,16852.0,1686.0,3533.0,NaN,16345.0,16 135,12603.0,3938.0,395.0,0.0,312.0,0.0,NaN,13200.0,0.0,2022-11-22 02:00:00+00:00
3,2022-11-22 00:00:00,3,16802.0,1689.0,3585.0,NaN,16345.0,16 135,12550.0,3996.0,452.0,0.0,256.0,0.0,NaN,13165.0,0.0,2022-11-22 03:00:00+00:00
4,2022-11-22 00:00:00,4,16837.0,1680.0,3894.0,NaN,16345.0,16 135,12241.0,4180.0,635.0,0.0,416.0,0.0,NaN,13136.0,0.0,2022-11-22 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,2023-03-26 00:00:00,20,16808.0,1480.0,4306.0,NaN,18717.0,13 304,8640.0,6051.0,2260.0,0.0,2118.0,358.0,NaN,12266.0,0.0,2023-03-26 20:00:00+00:00
2997,2023-03-26 00:00:00,21,15890.0,1438.0,4662.0,NaN,18774.0,13 003,8341.0,5895.0,2187.0,0.0,1654.0,0.0,NaN,11909.0,0.0,2023-03-26 21:00:00+00:00
2998,2023-03-26 00:00:00,22,15237.0,1340.0,5863.0,NaN,18965.0,13 212,7349.0,5858.0,2166.0,0.0,2031.0,0.0,NaN,10765.0,0.0,2023-03-26 22:00:00+00:00
2999,2023-03-26 00:00:00,23,14470.0,1282.0,6001.0,NaN,18555.0,13 011,7009.0,5541.0,1836.0,0.0,1919.0,0.0,NaN,9853.0,0.0,2023-03-26 23:00:00+00:00


In [63]:
df_2.index = df_2.index + 1

In [64]:
df = pd.concat([df_1, df_2])

In [65]:
df['Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4)'].fillna(0.0, inplace=True)
df['Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB'].fillna(0.0, inplace=True)

In [66]:
# teraz uśrednijmy nany rekordem przed i za
df.fillna(df.fillna(method='bfill'),inplace=True)

In [67]:
df.isnull().sum()

Doba                                                                                                                                                                                    0
Godzina                                                                                                                                                                                 0
Prognozowane zapotrzebowanie sieci                                                                                                                                                      0
Wymagana rezerwa mocy OSP                                                                                                                                                               0
Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13)                                                                                                                          0
Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4)   

In [68]:
# teraz połączenie
df.index = df['Doba_2']


In [69]:
# ================================================================================================================================================================================================

In [70]:
df_price = pd.read_csv('data\\ML_Intern_data.csv')

In [71]:
df_price = df_price.set_index('date')

In [72]:
df_price.index = pd.to_datetime(df_price.index)

In [73]:
df_concat = df.merge(df_price, how='left', left_index=True, right_index=True)

In [74]:
df_concat

,Doba,Godzina,Prognozowane zapotrzebowanie sieci,Wymagana rezerwa mocy OSP,Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13),Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4),Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB dostêpna dla OSP,Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9),Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB,Prognozowana sumaryczna generacja róde³ wiatrowych,Prognozowana sumaryczna generacja róde³ fotowoltaicznych,Planowane saldo wymiany miêdzysystemowej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Przewidywana generacja zasobów wytwórczych nieobjêtych obowi¹zkami mocowymi,Obowi¹zki mocowe wszystkich jednostek rynku mocy,Doba_2,da1_price
Doba_2,,,,,,,,,,,,,,,,,,,
2022-11-22 00:00:00+00:00,2022-11-22 00:00:00,0,18031.0,1741.0,5167.0,0.0,18955.0,18 745,13578.0,3982.0,346.0,0.0,471.0,0.0,0.0,14838.0,0.0,2022-11-22 00:00:00+00:00,870.00
2022-11-22 01:00:00+00:00,2022-11-22 00:00:00,1,16573.0,1652.0,3428.0,0.0,16082.0,15 872,12444.0,3883.0,322.0,0.0,247.0,0.0,0.0,13271.0,0.0,2022-11-22 01:00:00+00:00,830.00
2022-11-22 02:00:00+00:00,2022-11-22 00:00:00,2,16852.0,1686.0,3533.0,0.0,16345.0,16 135,12603.0,3938.0,395.0,0.0,312.0,0.0,0.0,13200.0,0.0,2022-11-22 02:00:00+00:00,800.00
2022-11-22 03:00:00+00:00,2022-11-22 00:00:00,3,16802.0,1689.0,3585.0,0.0,16345.0,16 135,12550.0,3996.0,452.0,0.0,256.0,0.0,0.0,13165.0,0.0,2022-11-22 03:00:00+00:00,789.00
2022-11-22 04:00:00+00:00,2022-11-22 00:00:00,4,16837.0,1680.0,3894.0,0.0,16345.0,16 135,12241.0,4180.0,635.0,0.0,416.0,0.0,0.0,13136.0,0.0,2022-11-22 04:00:00+00:00,820.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-07 19:00:00+00:00,2023-12-07 00:00:00,19,23526.0,2200.0,5092.0,2892.0,21990.0,21 990,16898.0,4931.0,721.0,0.0,1697.0,0.0,0.0,2608.0,23563.0,2023-12-07 19:00:00+00:00,590.00
2023-12-07 20:00:00+00:00,2023-12-07 00:00:00,20,23336.0,2211.0,5299.0,3087.0,22315.0,22 315,17016.0,4958.0,717.0,0.0,1362.0,0.0,0.0,2619.0,23563.0,2023-12-07 20:00:00+00:00,520.00
2023-12-07 21:00:00+00:00,2023-12-07 00:00:00,21,22674.0,2136.0,5942.0,3806.0,22315.0,22 315,16372.0,4865.0,735.0,0.0,1438.0,0.0,0.0,2607.0,23563.0,2023-12-07 21:00:00+00:00,440.00


In [75]:
df_concat.index.name = 'Time'

In [76]:
df_concat.fillna(method='bfill',inplace=True)

In [77]:
df_concat

,Doba,Godzina,Prognozowane zapotrzebowanie sieci,Wymagana rezerwa mocy OSP,Nadwy¿ka mocy dostêpna dla OSP (7) + (9) - [(3) - (12)] - (13),Nadwy¿ka mocy dostêpna dla OSP ponad wymagan¹ rezerwê moc (5) - (4),Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Moc dyspozycyjna JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB dostêpna dla OSP,Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9),Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB,Prognozowana sumaryczna generacja róde³ wiatrowych,Prognozowana sumaryczna generacja róde³ fotowoltaicznych,Planowane saldo wymiany miêdzysystemowej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej,Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB,Przewidywana generacja zasobów wytwórczych nieobjêtych obowi¹zkami mocowymi,Obowi¹zki mocowe wszystkich jednostek rynku mocy,Doba_2,da1_price
Time,,,,,,,,,,,,,,,,,,,
2022-11-22 00:00:00+00:00,2022-11-22 00:00:00,0,18031.0,1741.0,5167.0,0.0,18955.0,18 745,13578.0,3982.0,346.0,0.0,471.0,0.0,0.0,14838.0,0.0,2022-11-22 00:00:00+00:00,870.00
2022-11-22 01:00:00+00:00,2022-11-22 00:00:00,1,16573.0,1652.0,3428.0,0.0,16082.0,15 872,12444.0,3883.0,322.0,0.0,247.0,0.0,0.0,13271.0,0.0,2022-11-22 01:00:00+00:00,830.00
2022-11-22 02:00:00+00:00,2022-11-22 00:00:00,2,16852.0,1686.0,3533.0,0.0,16345.0,16 135,12603.0,3938.0,395.0,0.0,312.0,0.0,0.0,13200.0,0.0,2022-11-22 02:00:00+00:00,800.00
2022-11-22 03:00:00+00:00,2022-11-22 00:00:00,3,16802.0,1689.0,3585.0,0.0,16345.0,16 135,12550.0,3996.0,452.0,0.0,256.0,0.0,0.0,13165.0,0.0,2022-11-22 03:00:00+00:00,789.00
2022-11-22 04:00:00+00:00,2022-11-22 00:00:00,4,16837.0,1680.0,3894.0,0.0,16345.0,16 135,12241.0,4180.0,635.0,0.0,416.0,0.0,0.0,13136.0,0.0,2022-11-22 04:00:00+00:00,820.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-07 19:00:00+00:00,2023-12-07 00:00:00,19,23526.0,2200.0,5092.0,2892.0,21990.0,21 990,16898.0,4931.0,721.0,0.0,1697.0,0.0,0.0,2608.0,23563.0,2023-12-07 19:00:00+00:00,590.00
2023-12-07 20:00:00+00:00,2023-12-07 00:00:00,20,23336.0,2211.0,5299.0,3087.0,22315.0,22 315,17016.0,4958.0,717.0,0.0,1362.0,0.0,0.0,2619.0,23563.0,2023-12-07 20:00:00+00:00,520.00
2023-12-07 21:00:00+00:00,2023-12-07 00:00:00,21,22674.0,2136.0,5942.0,3806.0,22315.0,22 315,16372.0,4865.0,735.0,0.0,1438.0,0.0,0.0,2607.0,23563.0,2023-12-07 21:00:00+00:00,440.00


In [78]:
# Now, let's uncecessary field plus those I think is kind of redundant: [1,2,3,4,5,13,14]
df_concat = df_concat.drop(['Doba', 'Godzina', 'Doba_2'], axis = 1)

In [79]:
df_concat.drop(df_concat.iloc[:, [1,2,3,4,5,13,14]], inplace=True, axis=1)

In [80]:
df_concat.isna().sum()

Prognozowane zapotrzebowanie sieci                                                                                                                                                      0
Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9)                                                                                     0
Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB                                                                                          0
Prognozowana sumaryczna generacja róde³ wiatrowych                                                                                                                                     0
Prognozowana sumaryczna generacja róde³ fotowoltaicznych                                                                                                                               0
Planowane saldo wymiany miêdzysystemowej                              

In [81]:
df_concat.index = pd.to_datetime(df_concat.index)

In [82]:
# Let's add variables for seasonality - day of week, hour, month (in winter there is colder)
df_concat["day_of_week"] = df_concat.index.dayofweek
df_concat["month"] = df_concat.index.month
df_concat["hour"] = df_concat.index.hour

In [83]:
# let's save our result to a csv file. We will use it later on to train our model multiple times
df_concat['Niedyspozycyjnosc'] = df_concat['Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej'] + df_concat['Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB']

In [84]:
df_concat.drop(['Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹ca z ograniczeñ sieciowych wystêpuj¹cych w sieci przesy³owej oraz sieci dystrybucyjnej w zakresie dostarczania energii elektrycznej','Prognozowana wielkoæ niedyspozycyjnoci wynikaj¹cych z warunków eksploatacyjnych JW wiadcz¹cych us³ugi bilansuj¹ce w ramach RB'], inplace=True, axis=1)

In [85]:
df_concat['zrodla_odnawialne'] = df_concat['Prognozowana sumaryczna generacja róde³ wiatrowych'] + df_concat['Prognozowana sumaryczna generacja róde³ fotowoltaicznych']
df_concat.drop(['Prognozowana sumaryczna generacja róde³ wiatrowych','Prognozowana sumaryczna generacja róde³ fotowoltaicznych'], inplace=True, axis=1)

In [86]:
df_concat['generacjaJW'] = df_concat['Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9)'] + df_concat['Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB']
df_concat.drop(['Przewidywana generacja JW i magazynów energii wiadcz¹cych us³ugi bilansuj¹ce w ramach RB (3) - (9)','Prognozowana generacja JW i magazynów energii nie wiadcz¹cych us³ug bilansuj¹cych w ramach RB'], inplace=True, axis=1)

In [87]:
df_concat.rename({'Planowane saldo wymiany miêdzysystemowej':'saldo'}, inplace=True, axis=1)

In [88]:
order = ['Prognozowane zapotrzebowanie sieci', 'generacjaJW', 'zrodla_odnawialne','Niedyspozycyjnosc','saldo', 'month', 'day_of_week','hour','da1_price']
df_concat = df_concat[order]

In [89]:
df_concat['hour'] += 1

In [90]:
df_concat

,Prognozowane zapotrzebowanie sieci,generacjaJW,zrodla_odnawialne,Niedyspozycyjnosc,saldo,month,day_of_week,hour,da1_price
Time,,,,,,,,,
2022-11-22 00:00:00+00:00,18031.0,17560.0,346.0,0.0,471.0,11,1,1,870.00
2022-11-22 01:00:00+00:00,16573.0,16327.0,322.0,0.0,247.0,11,1,2,830.00
2022-11-22 02:00:00+00:00,16852.0,16541.0,395.0,0.0,312.0,11,1,3,800.00
2022-11-22 03:00:00+00:00,16802.0,16546.0,452.0,0.0,256.0,11,1,4,789.00
2022-11-22 04:00:00+00:00,16837.0,16421.0,635.0,0.0,416.0,11,1,5,820.00
...,...,...,...,...,...,...,...,...,...
2023-12-07 19:00:00+00:00,23526.0,21829.0,721.0,0.0,1697.0,12,3,20,590.00
2023-12-07 20:00:00+00:00,23336.0,21974.0,717.0,0.0,1362.0,12,3,21,520.00
2023-12-07 21:00:00+00:00,22674.0,21237.0,735.0,0.0,1438.0,12,3,22,440.00


In [91]:
df_concat.to_csv('ready_dataset_ready.csv')